# 6 - REINFORCEMENT LEARNING

In [1]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn; seaborn.set()

In [9]:
dataset = pd.read_csv('Part 6 - Reinforcement Learning/Section 32 - Upper Confidence Bound (UCB)/Ads_CTR_Optimisation.csv') #name of the dataset

# the data set here is a simulation -> each row represents a user and the 1s in the row are the ads on which the user WOULD click. We want to determine as fast as possible the ads with the best ctr

In [12]:
#implementation of the UCB algorithm
import math
num_of_slection = [0]*d #intialize a list of 10 0s
sums_of_rewards = [0]*d
avg_reward = 0
tot_reward = 0
d = 10
N = 10000
ads_selected=[]

for i in range(0,N,1):
    ad = 0
    max_UCB = 0
    for j in range(0,d):
        if(num_of_slection[j] != 0):
            avg_reward =  sums_of_rewards[j]/num_of_slection[j]
            delta = math.sqrt(3/2 * math.log(i+1)/ num_of_slection[j])
            UCB = avg_reward + delta
        else:
            UCB = 1e400 #force selection of the ad
        if(UCB > max_UCB):
            max_UCB =UCB
            ad = j
    ads_selected.append(ad)
    num_of_slection[ad] += 1
    sums_of_rewards[ad] += dataset.values[i,ad]

        

## 6.1 - Multi Armed Bandit Problem and Upper Confidence Bound

Reinforcement Learning is a method to teach a machine to perform task by providing positive feedback for actions performed correctly. The Multi armed Bandit problem is the following:

Let's consider a slot machine with a lever which we need to pull in order to play. Say there are a set of such machines under the assumption that each of these machine has a distribution that allows the machine to pick the numbers. We have 5 machines and possibily there are 5 distributions. And wee need to decide which of these slot machine is the best for us.

Formally:

1 - We have $d$ arms, say ads, that we display to users when they connect to a web page

2-  Each user conncetion makes a 'round of game'

3- at each round we display only one ad

4- If the user clicks on the ad $\rightarrow$ we get the reward 1 otherwise we get the reward 0

5- Goal is to maximise the number of rewards

Upper bound confidence works as follows:

1- At each round $n$ for each ad $i$ there are two numbers associated: $N_i(n)$ the number of times the ad $i$ was selected up to round $n$, and $R_i(n)$ the sum of rewards up to round $n$ for ad $i$

2 - we can compute then 
$$ <r_i(n)> = \frac{R_i(n)}{N_i(n)} $$ 
the average reward and the confidence interval $\Delta_i(n)$
$$ \Delta_i(n) = \sqrt{ \frac{3 \log(n)}{2 N_i(n)} } $$

3 - we select the $i$ that has the highest sum $<r_i(n)> + \Delta_i(n)$


